<a href="https://colab.research.google.com/github/SanghaviHarshPankajkumar/Movie_Recommendation_System/blob/master/mini_project_Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Recommendations System

### import all Require Libraries

In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import nltk
import pickle

### import Dataset

In [6]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [7]:
!kaggle datasets download -d juzershakir/tmdb-movies-dataset

  0% 0.00/2.97M [00:00<?, ?B/s]
100% 2.97M/2.97M [00:00<00:00, 220MB/s]


In [8]:
from zipfile import ZipFile
dataset = '/content/tmdb-movies-dataset.zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('done...')

done...


### exploring dataset

In [9]:
movies_df  = pd.read_csv('/content/tmdb_movies_data.csv')

In [10]:
movies_df.head()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,5562,6.5,2015,137999939.3,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,6185,7.1,2015,137999939.3,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,2480,6.3,2015,101199955.5,2.716190e+08
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,5292,7.5,2015,183999919.0,1.902723e+09
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,2947,7.3,2015,174799923.1,1.385749e+09


In [12]:
movies_df.shape

(10866, 21)

In [13]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10866 entries, 0 to 10865
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    10866 non-null  int64  
 1   imdb_id               10856 non-null  object 
 2   popularity            10866 non-null  float64
 3   budget                10866 non-null  int64  
 4   revenue               10866 non-null  int64  
 5   original_title        10866 non-null  object 
 6   cast                  10790 non-null  object 
 7   homepage              2936 non-null   object 
 8   director              10822 non-null  object 
 9   tagline               8042 non-null   object 
 10  keywords              9373 non-null   object 
 11  overview              10862 non-null  object 
 12  runtime               10866 non-null  int64  
 13  genres                10843 non-null  object 
 14  production_companies  9836 non-null   object 
 15  release_date       

## Preprocessing

In [14]:
# require Features:- id, original_title, cast 3 only, diretor, keywords, overview, genres, release_year

movies_df = movies_df[['id','original_title','cast','director','keywords','overview','genres','release_year']]

In [15]:
movies_df.head()

,id,original_title,cast,director,keywords,overview,genres,release_year
0,135397,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,monster|dna|tyrannosaurus rex|velociraptor|island,Twenty-two years after the events of Jurassic ...,Action|Adventure|Science Fiction|Thriller,2015
1,76341,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,future|chase|post-apocalyptic|dystopia|australia,An apocalyptic story set in the furthest reach...,Action|Adventure|Science Fiction|Thriller,2015
2,262500,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,based on novel|revolution|dystopia|sequel|dyst...,Beatrice Prior must confront her inner demons ...,Adventure|Science Fiction|Thriller,2015
3,140607,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,android|spaceship|jedi|space opera|3d,Thirty years after defeating the Galactic Empi...,Action|Adventure|Science Fiction|Fantasy,2015
4,168259,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,car race|speed|revenge|suspense|car,Deckard Shaw seeks revenge against Dominic Tor...,Action|Crime|Thriller,2015


In [16]:
movies_df.shape

(10866, 8)

In [18]:
# null values
movies_df.isnull().sum()
movies_df.dropna(inplace = True)

In [20]:
# duplicate values
movies_df.duplicated().sum()
movies_df.drop_duplicates()
movies_df.reset_index(inplace = True)

In [21]:
def get_main_cast(text):
  casts =  text.split('|')
  final_cast = []
  count=0
  for i in casts:
    if count<3:
      final_cast.append(i)
      count+=1
    else:
      break

  return final_cast

def convert_into_list(text):
  final_list = text.split('|')
  return final_list

In [22]:
#cast 3 fetch
movies_df['cast'] = movies_df['cast'].apply(get_main_cast)

In [23]:
#genres convert in list
movies_df['genres'] = movies_df['genres'].apply(convert_into_list)

In [24]:
movies_df['keywords'] = movies_df['keywords'].apply(convert_into_list)

In [25]:
movies_df['overview'] = movies_df['overview'].apply(lambda x: x.split())

In [26]:
movies_df.head()

,index,id,original_title,cast,director,keywords,overview,genres,release_year
0,0,135397,Jurassic World,"[Chris Pratt, Bryce Dallas Howard, Irrfan Khan]",Colin Trevorrow,"[monster, dna, tyrannosaurus rex, velociraptor...","[Twenty-two, years, after, the, events, of, Ju...","[Action, Adventure, Science Fiction, Thriller]",2015
1,1,76341,Mad Max: Fury Road,"[Tom Hardy, Charlize Theron, Hugh Keays-Byrne]",George Miller,"[future, chase, post-apocalyptic, dystopia, au...","[An, apocalyptic, story, set, in, the, furthes...","[Action, Adventure, Science Fiction, Thriller]",2015
2,2,262500,Insurgent,"[Shailene Woodley, Theo James, Kate Winslet]",Robert Schwentke,"[based on novel, revolution, dystopia, sequel,...","[Beatrice, Prior, must, confront, her, inner, ...","[Adventure, Science Fiction, Thriller]",2015
3,3,140607,Star Wars: The Force Awakens,"[Harrison Ford, Mark Hamill, Carrie Fisher]",J.J. Abrams,"[android, spaceship, jedi, space opera, 3d]","[Thirty, years, after, defeating, the, Galacti...","[Action, Adventure, Science Fiction, Fantasy]",2015
4,4,168259,Furious 7,"[Vin Diesel, Paul Walker, Jason Statham]",James Wan,"[car race, speed, revenge, suspense, car]","[Deckard, Shaw, seeks, revenge, against, Domin...","[Action, Crime, Thriller]",2015


In [27]:
#remove spaces between words
def remove_space(text):
  final_list = []
  for i in text:
    final_list.append(i.replace(" ",""))

  return final_list

In [28]:
movies_df['cast'] = movies_df['cast'].apply(remove_space)
movies_df['keywords'] = movies_df['keywords'].apply(remove_space)
movies_df['genres'] = movies_df['genres'].apply(remove_space)
movies_df['director'] = movies_df['director'].apply(lambda x: x.replace(" ","").split(" "))

In [29]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9306 entries, 0 to 9305
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   index           9306 non-null   int64 
 1   id              9306 non-null   int64 
 2   original_title  9306 non-null   object
 3   cast            9306 non-null   object
 4   director        9306 non-null   object
 5   keywords        9306 non-null   object
 6   overview        9306 non-null   object
 7   genres          9306 non-null   object
 8   release_year    9306 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 654.5+ KB


### creating tags from features

In [30]:
movies_df['tags'] = movies_df['cast'] + movies_df['overview']+movies_df['keywords']+movies_df['genres']+movies_df['director']

In [31]:
final_movies = movies_df[['id','original_title','release_year','tags']]

In [32]:
final_movies.head()

,id,original_title,release_year,tags
0,135397,Jurassic World,2015,"[ChrisPratt, BryceDallasHoward, IrrfanKhan, Tw..."
1,76341,Mad Max: Fury Road,2015,"[TomHardy, CharlizeTheron, HughKeays-Byrne, An..."
2,262500,Insurgent,2015,"[ShaileneWoodley, TheoJames, KateWinslet, Beat..."
3,140607,Star Wars: The Force Awakens,2015,"[HarrisonFord, MarkHamill, CarrieFisher, Thirt..."
4,168259,Furious 7,2015,"[VinDiesel, PaulWalker, JasonStatham, Deckard,..."


In [33]:
final_movies.shape

(9306, 4)

### Preprossess tags column

In [ ]:
final_movies['tags'] = final_movies['tags'].apply(lambda x: " ".join(x))

### natural language processing

In [35]:
nltk.download('stopwords')
nltk.download('punkt')
sw_list = stopwords.words('english')
word_stemmer = PorterStemmer()

def remove_tags(text):
  return re.sub(re.compile('<.*?>'), '', text)

def to_lower(text):
  return text.lower()

def remove_stop_words(text):
  word_tokens = word_tokenize(text)
  final_text  =" ".join([w for w in word_tokens if not w in sw_list])

  return final_text

def stemming(text):
  word_tokens = word_tokenize(text)
  final_text  =" ".join([word_stemmer.stem(w) for w in word_tokens])
  return final_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
final_movies['tags'] = final_movies['tags'].apply(remove_tags)
final_movies['tags'] = final_movies['tags'].apply(to_lower)
final_movies['tags'] = final_movies['tags'].apply(remove_stop_words)
final_movies['tags'] = final_movies['tags'].apply(stemming)

In [37]:
final_movies['tags']

0       chrispratt brycedallashoward irrfankhan twenty...
1       tomhardi charlizetheron hughkeays-byrn apocaly...
2       shailenewoodley theojam katewinslet beatric pr...
3       harrisonford markhamil carriefish thirti year ...
4       vindiesel paulwalk jasonstatham deckard shaw s...
                              ...                        
9301    michaelhynson robertaugust lord'tallyho'blear ...
9302    jamesgarn evamariesaint yvesmontand grand prix...
9303    innokentiysmoktunovskiy olegefremov georgizhzh...
9304    tatsuyamihashi akikowakabayashi miehama comic ...
9305    haroldp.warren tomneyman johnreynold famili ge...
Name: tags, Length: 9306, dtype: object

## Model Creation

### Making Vectors Using CountVectorizer

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
cVec = CountVectorizer(max_features=5000)

In [39]:
vectors = cVec.fit_transform(final_movies['tags']).toarray()

In [40]:
vectors.shape

(9306, 5000)

### Creating Similarity Matrix using Cosine Similarity

In [41]:
from sklearn.metrics.pairwise import cosine_similarity

In [42]:
similarity = cosine_similarity(vectors)

In [43]:
similarity

array([[1.        , 0.18887687, 0.13650473, ..., 0.        , 0.07049074,
        0.        ],
       [0.18887687, 1.        , 0.12354155, ..., 0.        , 0.13397283,
        0.        ],
       [0.13650473, 0.12354155, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.02942449,
        0.        ],
       [0.07049074, 0.13397283, 0.        , ..., 0.02942449, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

### evaluating model

In [44]:
def recommend(movie):
    index = final_movies[final_movies['original_title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(final_movies.iloc[i[0]].original_title)

In [47]:
recommend('Superman')

Superman II
All Star Superman
Superman Returns
Superman IV: The Quest for Peace
Supergirl


## Exporting Model and require data

In [49]:
pickle.dump(final_movies,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))